In [1]:
from googleapiclient.discovery import build

# Reading a key from *.key.txt
def readKey(filename):
    key = None
    try:
        f = open(filename, "r")
        contents = f.read()
        if (contents == ''):
            raise
        else:
            key = contents
    except:
        print("Please paste your API key in '" + filename + "' file")
        f = open(filename, "w")
    f.close()
    return key

In [16]:
from pymongo import MongoClient

class DB:
    def __init__(self, key, database):
        self.collection = 'videos'
        self.client = MongoClient(key)
        self.db = self.client[database]
        
    def addVideo(self, id, payload):
        self.db[self.collection].update_one({'_id': id}, {"$set": payload}, upsert=True)
        
    def getAllVideos(self):
        res = self.db[self.collection].find()
        return [x for x in res]
    
    def clearCollection(self, msg=''):
        if (msg == 'yes'):
            self.db[self.collection].delete_many({})
            print("Colelction was cleared.")
        else:
            print("Pass 'yes' into clearCollection() method to really clear it.")
            
    def close(self):
        self.client.close()

In [3]:
class YouTube:
    def __init__(self, KEY, db=None):
        self._YOUTUBE_API_SERVICE_NAME = 'youtube'
        self._YOUTUBE_API_VERSION = 'v3'
        self._KEY = KEY
        self.youtube = build(self._YOUTUBE_API_SERVICE_NAME, self._YOUTUBE_API_VERSION, 
                             developerKey=self._KEY)
        
        self.lastSearch = None
        self.results = []
        
        self.db = db
    
    # Uses youtube.search() API method
    def search(self, q, limit=1, details=False, links=False, videoOnly=False, save=False):
        ''' 
        q - search query, (ex. "Whale Shark")
        limit - number of results (ex. 10 or 100)
        details - Retrieves more detailed information about the video (ex. True, False)
                  Cannot be True when videoOnly=False.
        links - Generate YouTube video links for each search result. 
                Can be found in video['url'] (ex. True, False) 
        videosOnly - Return only videos from the result (ex. True, False)
        save - Saves all retrieved videos to database (ex. True, False)
        '''
        if (details and not videoOnly):
            return print('You cannot use details=True and videoOnly=False together. But any other combination is valid.')
        
        if (save):
            if (not self.db):
                save = False
                print("Please provide 'db' argument with an instance to database to save video(s).")
        
        # Quering the result
        searchResult = self.youtube.search().list(
            q=q,
            part='snippet',
            fields='items(id,snippet(publishedAt,title,description))',
            type='video' if videoOnly else 'video,channel,playlist',
            maxResults=limit
        ).execute()
        self.lastSearch = searchResult['items']
                
        
        # Handling additional parameters
        if (details or save or links):
            for item in self.lastSearch:
                
                # Quering more details for this result
                if (details):
                    details = self.videos(item['id']['videoId'])
                    for prop in details[0]:
                        try:
                            item[prop].update(details[0][prop])
                        except KeyError:
                            item[prop] = details[0][prop]
                            
                # Generating YouTube video links
                if (links):
                    if (item['id']['kind'] == 'youtube#video'):
                        item['url'] = 'https://youtu.be/' + item['id']['videoId']
                    elif (item['id']['kind'] == 'youtube#channel'):
                        item['url'] = 'https://www.youtube.com/channel/' + item['id']['channelId']

                # Saving item in database
                if (save):
                    self.db.addVideo(item['id']['videoId'], item)
                    

        # Appeding result to all previous search results
        self.results = self.lastSearch + self.results
        return self.lastSearch
    
    # Uses youtube.videos() API method
    def videos(self, id, limit=1, links=False):
        searchResult = self.youtube.videos().list(
            part='snippet,statistics',
            fields='items(snippet(tags),statistics)',
            id=id
        ).execute()
        
        return searchResult['items']

In [17]:
DB_KEY = readKey("db.key.txt")
db = DB(DB_KEY, 'youtube')

YT_KEY = readKey('yt.key.txt')
yt = YouTube(YT_KEY, db)

In [14]:
yt.search('tiger', limit=1, details=False, links=True, videoOnly=True, save=False)

[{'id': {'kind': 'youtube#video', 'videoId': '_UbDeqPdUek'},
  'snippet': {'publishedAt': '2015-08-26T12:16:58.000Z',
   'title': 'Cubs Meet Adult Tiger for the First Time | Tigers About The House | BBC Earth',
   'description': "As the cubs grow up Giles Clark thinks it's time for Spot and Stripe to start socialising with the adult tigers. Taken from Tigers About The House. Subscribe: ..."},
  'url': 'https://youtu.be/_UbDeqPdUek'}]

In [15]:
db.getAllVideos()

[{'_id': 'bNYXweQ81vI',
  'id': {'kind': 'youtube#video', 'videoId': 'bNYXweQ81vI'},
  'snippet': {'publishedAt': '2015-01-14T00:17:13.000Z',
   'title': 'Giraffes for Kids: Learn about Giraffes - FreeSchool',
   'description': "https://patreon.com/freeschool - Help support more content like this! Let's take a look at giraffes and learn a little more about this familiar yet exotic animal."},
  'url': 'https://youtu.be/bNYXweQ81vI'},
 {'_id': 'E9tpPWK7sag',
  'id': {'kind': 'youtube#video', 'videoId': 'E9tpPWK7sag'},
  'snippet': {'publishedAt': '2017-01-24T13:00:05.000Z',
   'title': 'Investigating the Mysterious Whale Sharks of Mafia Island | National Geographic',
   'description': "The whale sharks of Mafia Island are unusual because they don't migrate—and researchers want to know why. ➡ Subscribe: http://bit.ly/NatGeoSubscribe About ..."},
  'url': 'https://youtu.be/E9tpPWK7sag'},
 {'_id': '8U48NfAEEGQ',
  'id': {'kind': 'youtube#video', 'videoId': '8U48NfAEEGQ'},
  'snippet': {'pub

In [9]:
# db.clearCollection('yes')

In [18]:
db.close()